# Imports

In [ ]:
import sklearn.model_selection as skm
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import (DecisionTreeClassifier as DTC, DecisionTreeRegressor as DTR, plot_tree, export_text)
from sklearn.metrics import (accuracy_score, log_loss)
from sklearn.ensemble import (RandomForestRegressor as RF, GradientBoostingRegressor as GBR)
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
import numpy as np
import pandas as pd
from sklearn import tree
import csv
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import (DecisionTreeClassifier as DTC, DecisionTreeRegressor as DTR, plot_tree, export_text)
from sklearn.metrics import (accuracy_score, log_loss)
from sklearn.ensemble import (RandomForestRegressor as RF, GradientBoostingRegressor as GBR)
from scipy.stats import loguniform, randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
from scipy.stats import randint, uniform, loguniform
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
import smogn
import os
from sklearn.svm import SVR
import concurrent.futures
import optuna

# Data Prep

In [ ]:
recovery_balanced = pd.read_csv('Resampling_output_data/unbalanced.csv')
X_test = pd.read_csv('Resampling_output_data/X_test_unbalanced.csv')
y_test = pd.read_csv('Resampling_output_data/y_test_unbalanced.csv')

In [ ]:
print(pd.DataFrame(y_test).value_counts())
print(pd.DataFrame(y_test).value_counts(normalize=True)) 

In [ ]:
recovery_balanced_train = recovery_balanced.drop(columns = ['Unnamed: 0'])

In [ ]:
Duration = recovery_balanced_train['Duration_int']
Recovery_rate = recovery_balanced_train['Actual_Recovery_Rate_label']
X_train = recovery_balanced_train.drop(columns = ['Duration_int','Actual_Recovery_Rate_label','Expected_Recoveries','Actual_Recoveries_label'])

In [ ]:
X_train = pd.get_dummies(X_train, columns = X_train.columns)
X_train['Duration'] = Duration

In [ ]:
y_train = Recovery_rate

In [ ]:
Duration = X_test['Duration_int']
X_test = X_test.drop(columns = ['Unnamed: 0','Duration_int','Expected_Recoveries','Actual_Recovery_Rate_label'])

In [ ]:
X_test = pd.get_dummies(X_test, columns = X_test.columns)
X_test['Duration'] = Duration

In [ ]:
X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)

In [ ]:
X_test.shape

In [ ]:
y_test = y_test.drop(columns = 'Unnamed: 0')

In [ ]:
y_test = np.array(y_test)

In [ ]:
__ = pd.concat([X_train, y_train], axis = 1)
__.reset_index(inplace=True)
_ = __.sample(frac = 1)
_.reset_index(inplace=True)
y_train = _['Actual_Recovery_Rate_label']
X_train = _.drop(columns = ['Actual_Recovery_Rate_label','level_0','index'])

In [ ]:
print(min(y_train),max(y_train))

# Modelling

## Optuna

In [ ]:
from optuna.terminator.callback import TerminatorCallback
from optuna.terminator import report_cross_validation_scores
from optuna.trial import TrialState
import time

In [ ]:
optuna_db_path = "sqlite:////Users/mingyanggao/Desktop/SRF/my_optuna_study.db"

optuna-dashboard sqlite:////Users/mingyanggao/Desktop/SRF/my_optuna_study.db

## Past Performance

MSE from the DTR of the provided paper

In [ ]:
from sklearn.metrics import root_mean_squared_error

In [ ]:
model_DTR_pre = DTR(max_depth = 15, min_samples_split = 110, ccp_alpha = 0.0000001)
model_DTR_pre.fit(X_train, y_train)

In [ ]:
# Ensure y_test is a 1D numpy array of the same length as X_test
if isinstance(y_test, pd.DataFrame) or isinstance(y_test, pd.Series):
    y_test = y_test.values.flatten()
if hasattr(y_test, 'shape') and len(y_test.shape) > 1:
    y_test = y_test.flatten()
if hasattr(y_test, 'reset_index'):
    y_test = pd.Series(y_test).reset_index(drop=True).values

In [ ]:
y_pred = model_DTR_pre.predict(X_test)
root_mean_squared_error(y_pred,y_test)

# Summary and Nonparametric

In [ ]:
#NN_score = test_loss.item() 
CatBoost_score = 3.3276566434721118
XGBoost_score = 3.335625247106174 
Pre_score = 3.3285080299643295

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

do the catboost extraction of pre-coded data first

In [ ]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 1)
cv_previous = []
cv_cat = []
cv_XGBoost = []
X_test.reset_index(drop=True, inplace=True)

for train_idx, test_idx in kf.split(X_test, y_test):
    X_test_batch = X_test.iloc[test_idx]
    y_test_batch = y_test[test_idx]

    a = root_mean_squared_error(model_DTR_pre.predict(X_test_batch),y_test_batch)
    cv_previous.append(a)
   

for train_idx, test_idx in kf.split(X_test_, y_test):
    X_test_batch = X_test_.iloc[test_idx]
    y_test_batch = y_test[test_idx]

    test_pool = Pool(data = X_test_batch, cat_features = ['AgeBand_cat', 'Disability_cat', 'OwnOcc_cat', 'STD_cat', 'Gender_bin_cat', 'Benefit_amount_cat'])

    dtest = xgb.DMatrix(
      X_test_batch, 
      enable_categorical=True)

    #catboost
    pred_cat = trial_model.predict(test_pool)
    rmse_cat = root_mean_squared_error(y_test_batch, pred_cat)

    #xgboost
    pred_xgb = bst.predict(dtest)
    rmse_xgboost = root_mean_squared_error(y_test_batch, pred_xgb)

    cv_cat.append(rmse_cat)
    cv_XGBoost.append(rmse_xgboost)

In [ ]:
cv_XGBoost

In [ ]:
#KFold CV with NN

cv_NN = list()
kf = KFold(n_splits=5, shuffle=True, random_state=42)
_ = torch.cat([X_train_tensor, y_train_tensor], dim=1)

for fold, (train_idx, val_idx) in enumerate(kf.split(_)):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    a = X_train.shape[1]
    X_tr_numeric = X_tr.astype(np.float32)
    X_val_numeric = X_val.astype(np.float32)

    X_tr_tensor = torch.tensor(X_tr_numeric.values, dtype=torch.float32)
    y_tr_tensor = torch.tensor(y_tr.values, dtype=torch.float32).view(-1, 1)

    X_val_tensor = torch.tensor(X_val_numeric.values, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.float32).view(-1, 1)

    X_tr_tensor = X_tr_tensor.to(DEVICE)
    y_tr_tensor = y_tr_tensor.to(DEVICE)
    X_val_tensor = X_val_tensor.to(DEVICE)
    y_val_tensor = y_val_tensor.to(DEVICE)

    for epoch in range(50):
        NN.train()
        optimizer.zero_grad()
        output = NN(X_tr_tensor)
        loss = criterion(output, y_tr_tensor)
        loss.backward()
        optimizer.step()

    NN.eval()
    with torch.no_grad():
        predictions = NN(X_val_tensor)
        val_loss = criterion(predictions, y_val_tensor)
        cv_NN.append(val_loss)

In [ ]:
cv_NN

In [ ]:
cv_NN_list = [13.3711, 16.2190, 15.1451, 13.6903, 12.8620]

In [ ]:
from scipy.stats import kruskal

In [ ]:
stat, p_value = kruskal(cv_previous, cv_XGBoost, cv_cat)
print(f"Kruskal-Wallis H-statistic: {stat}, p-value: {p_value}")

In [ ]:
_ = pd.DataFrame({'Models' : ['Pre','XGBoost','cat'],
              'Score':[Pre_score,XGBoost_score,CatBoost_score]})
sns.barplot(x='Models',y='Score',data = _)
plt.show()

In [ ]:
df = pd.DataFrame({'previous':cv_previous,
              'boost':cv_boost,
              'NN':cv_NN_list})
df

In [ ]:
import scikit_posthocs as sp

# Combine data into a DataFrame
data = list(cv_previous) + list(cv_boost) + cv_NN_list
groups = ['previous']*len(cv_previous) + ['boost']*len(cv_boost) + ['NN']*len(cv_NN_list) 
df_2 = pd.DataFrame({'group': groups, 'value': data})

# Perform Dunn's test with Bonferroni correction
posthoc = sp.posthoc_dunn(df_2, val_col='value', group_col='group', p_adjust='bonferroni')
print(posthoc)

In [ ]:
from scipy.stats import mannwhitneyu

stat, p = mannwhitneyu(list(cv_previous), list(cv_cat), alternative='two-sided')
print(f"Previous rMSEs: {list(cv_previous)}")
print(f"XGBoost rMSEs: {list(cv_cat)}")
print(f"Mann-Whitney U statistic: {stat}, p-value: {p}")

if p < 0.05:
    print("Difference in MSE is statistically significant.")
else:
    print("No significant difference in MSE.")

## Support Vector Regression

In [ ]:
def objective(trial):
        max_seconds = 60
        start_time = time.time()

        svr_kernel = trial.suggest_categorical('kernel', ['rbf', 'poly'])
        svr_shrink = trial.suggest_categorical('shrinking', [True, False])
        svr_C = trial.suggest_float('svr_rbf_C', 1e-3, 1e3, log=True)
        svr_epsilon = trial.suggest_float('epsilon', 0.1, 1, step=0.1)
        regressor_obj = SVR(kernel=svr_kernel, shrinking=svr_shrink, C=svr_C, epsilon=svr_epsilon, gamma='auto')

        #sample run
        #small_X, _, small_y, _ = train_test_split(X_train, y_train, test_size=0.5)
        #start_fit = time.time()
        #regressor_obj.fit(small_X, small_y)
        #fit_duration = time.time() - start_fit
        #if fit_duration > max_seconds:
        #    raise optuna.TrialPruned()

        scores = skm.cross_val_score(regressor_obj, X_train, y_train, n_jobs=-1, cv=3, scoring='neg_mean_squared_error')
        actual_mse_scores = [-s for s in scores]
        mean_mse = np.mean(actual_mse_scores)
        report_cross_validation_scores(trial, scores.tolist())
        return (mean_mse)


if __name__ == '__main__':
    study = optuna.create_study(direction='minimize', 
                            storage=optuna_db_path, 
                            study_name='SVR_5%', 
                            load_if_exists=True)
    study.optimize(objective,
                   n_trials= 50)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

##  Boosting

### CatBoost

#### pre-coded Data

In [ ]:
recovery_balanced_ = pd.read_csv('Resampling_output_data/unbalanced.csv')
X_test_ = pd.read_csv('Resampling_output_data/X_test_unbalanced.csv')
y_test_ = pd.read_csv('Resampling_output_data/y_test_unbalanced.csv')

recovery_balanced_train_ = recovery_balanced.drop(columns = ['Unnamed: 0'])
X_train_ = recovery_balanced_train.drop(columns = ['Actual_Recovery_Rate_label','Expected_Recoveries','Actual_Recoveries_label'])

y_train_ = Recovery_rate


X_test_ = X_test_.drop(columns = ['Unnamed: 0','Expected_Recoveries','Actual_Recovery_Rate_label'])
y_test_ = y_test_.drop(columns = 'Unnamed: 0')
y_test_ = np.array(y_test_)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_alt, X_val, y_train_alt, y_val = train_test_split(X_train_,y_train_,test_size=0.3)

In [ ]:
from catboost import Pool

In [ ]:
train_pool = Pool(
    data = X_train_alt, 
    label = y_train_alt,
    cat_features = ['AgeBand_cat', 'Disability_cat', 'OwnOcc_cat',
       'STD_cat', 'Gender_bin_cat', 'Benefit_amount_cat'])

valid_pool = Pool(
    data = X_val, 
    label = y_val,
    cat_features = ['AgeBand_cat', 'Disability_cat', 'OwnOcc_cat',
       'STD_cat', 'Gender_bin_cat', 'Benefit_amount_cat'])
       
test_pool = Pool(
    data = X_test_, 
    cat_features = ['AgeBand_cat', 'Disability_cat', 'OwnOcc_cat',
    'STD_cat', 'Gender_bin_cat', 'Benefit_amount_cat'])

report_pool = Pool(
    data = X_train_,
     label = y_train_,
    cat_features = ['AgeBand_cat', 'Disability_cat', 'OwnOcc_cat',
    'STD_cat', 'Gender_bin_cat', 'Benefit_amount_cat'])

#### Mode Using Optuna

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
def objective(trial):
        param = {
        "thread_count": -1,   
        'grow_policy':'Lossguide',
        'iterations': 100000,
        'learning_rate': trial.suggest_float("learning_rate", 0.1, 1),
        "depth" : trial.suggest_int("depth", 5,20),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 25, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1),
        #"bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli"]),
        "max_leaves": trial.suggest_int("max_leaves", 20, 100),
        'od_type':"Iter",
        'od_wait':20
        }

        trial_model  = CatBoostRegressor(**param).fit(
            train_pool,
            eval_set = valid_pool
        )
        
        pred = trial_model.predict(test_pool)
        rmse = root_mean_squared_error(y_test, pred)
        
        trial.set_user_attr("best_iter", trial_model.get_best_iteration())
        return rmse

if __name__ == "__main__":
    #sampler = TPESampler()
    study = optuna.create_study(#sampler = sampler,
                                storage=optuna_db_path,
                                study_name='Final_code_CatBoost_Optuna',
                                load_if_exists=True,
                                direction="minimize")

    study.optimize(objective, n_trials=20000)

    print("Number of finished5rt6 trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

### Model Without Optuna

In [ ]:
trial_model  = CatBoostRegressor(
            od_wait = 50,
            use_best_model=True
).fit(
            train_pool,
            eval_set = valid_pool
        )

In [ ]:
pred = trial_model.predict(test_pool)
rmse = root_mean_squared_error(y_test, pred)
print(rmse)

### XGBoost

In [ ]:
X_train_.columns

In [ ]:
X_train_duration = X_train_['Duration_int']
X_train = X_train_.drop(columns = ['Duration_int'])
X_test_duration = X_test_['Duration_int']
X_test  = X_test_.drop(columns = ['Duration_int'])

X_train_.columns  = X_train_.columns.astype(str)
X_test_.columns   = X_test_.columns.astype(str)
X_train_.columns  = X_train_.columns.str.replace(r'[\$ ,)-]', '_', regex=True)
X_test_.columns   = X_test_.columns.str.replace(r'[\$ ,)-]', '_', regex=True)

X_train_['Duration_int'] =  X_train_duration
X_test_['Duration_int']  =  X_test_duration

In [ ]:
for x in ['AgeBand_cat', 'Disability_cat', 'OwnOcc_cat', 'STD_cat',
       'Gender_bin_cat', 'Benefit_amount_cat']:
       X_train_[x] = X_train_[x].astype('category')
       X_test_[x] = X_test_[x].astype('category')

In [ ]:
X_train_alt, X_val, y_train_alt, y_val = train_test_split(X_train_,y_train_,test_size=0.3)

In [ ]:
dtrain = xgb.DMatrix(
    X_train_alt, 
    label = y_train_alt,
    enable_categorical=True)

dvalid = xgb.DMatrix(
    X_val, 
    label = y_val,
    enable_categorical=True)
       
dtest = xgb.DMatrix(
    X_test_, 
    enable_categorical=True)

dreport = xgb.DMatrix(
    X_train_,
    label = y_train,
    enable_categorical=True)

In [ ]:
params = {
    "booster": "gbtree",
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "learning_rate": 0.1,
    "max_depth": 6,
    "min_child_weight": 1,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "gamma": 5,
    "lambda": 1,
    "alpha": 1,
    "tree_method": "hist"
}


In [ ]:
dtrain.num_row(), dvalid.num_row(), dtest.num_row()

In [ ]:
def objective(trial):
    param = {
        "booster": "gbtree",
        "verbosity": 1,
        "eval_metric": 'rmse',
        "learning_rate": trial.suggest_float('learning_rate',0.01,0.04),
        "max_depth": trial.suggest_int('max_depth',1,5),
        "subsample": trial.suggest_float('subsample',0.5,1),
        "gamma":  trial.suggest_float('gamma',1,20),
        "lambda": trial.suggest_float('gamma',1,20),
        "alpha": trial.suggest_float('gamma',1,20),
        "tree_method": "hist"
    }

    model = xgb.train(
    param,
    dtrain = dtrain,
    num_boost_round=5000,
    evals=[(dvalid, 'valid')],
    early_stopping_rounds=10
    )

    bst = xgb.train(
    param,
    dtrain = dtrain,
    num_boost_round= model.best_iteration
    #evals=[(dvalid, 'valid')],
    #early_stopping_rounds=50
    )

    y_pred = bst.predict(dtest)
    rmse_test = root_mean_squared_error(y_test,y_pred)
    trial.set_user_attr('iter',model.best_iteration)

    return rmse_test
    
if __name__ == "__main__":
    sampler = TPESampler()
    study = optuna.create_study(
                                sampler = sampler,
                                storage=optuna_db_path,
                                study_name='SRF_XGBoost_UPdate',
                                load_if_exists=True,
                                direction="minimize")

    study.optimize(objective, n_trials=20000)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
model = xgb.train(
    params,
    dtrain = dtrain,
    num_boost_round=5000,
    evals=[(dvalid, 'valid')],
    early_stopping_rounds=50
)

In [ ]:
bst = xgb.train(
    params,
    dtrain = dreport,
    num_boost_round= model.best_iteration
    #evals=[(dvalid, 'valid')],
    #early_stopping_rounds=50
)

In [ ]:
y_pred_xgboost = bst.predict(dtest)
rmse_xgboost = root_mean_squared_error(y_test,y_pred_xgboost)
print(rmse_xgboost)

#### without pruning

In [ ]:
from optuna.samplers import RandomSampler, TPESampler, GridSampler, CmaEsSampler
from sklearn.metrics import mean_squared_error
import xgboost

In [ ]:
def objective(trial):
        param = {
        "verbosity": 1,
        "eval_metric": 'rmse',
        "booster": trial.suggest_categorical("booster", ['gbtree','dart']),
        "lambda": trial.suggest_float("lambda", 0.5, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 0.5, 1.0, log=True),
        }


        param["max_depth"] = trial.suggest_int("max_depth", 1, 5)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 0.5, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
        
        #sample_type = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        #normalize_type = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        #rate_drop = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        #skip_drop = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
        
        #if param['booster'] == 'dart':
        #    param["sample_type"] = sample_type
        #    param["normalize_type"] = normalize_type
            #param["rate_drop"] = rate_drop
            #param["skip_drop"] = skip_drop

        model = xgb.train(
                param,
                dtrain = dtrain,
                num_boost_round=500,
                evals=[(dvalid, 'valid')],
                early_stopping_rounds=20
            )
        
        final_model = xgb.train(
            param,
            dtrain = dreport,
            num_boost_round = model.best_iteration
        )

        y_pred = final_model.predict(dtest)
        rmse_trial = root_mean_squared_error(y_test,y_pred)
        trial.set_user_attr('iter',model.best_iteration)

        return rmse_trial

if __name__ == "__main__":
    sampler = TPESampler()
    study = optuna.create_study(
                                sampler = sampler,
                                storage=optuna_db_path,
                                study_name='SRF_xgboost_update_new_version',
                                load_if_exists=True,
                                direction="minimize")

    study.optimize(objective, n_trials=20000)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
def objective(trial):
    #try:
        param = {
        "learning_rate": trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        "tree_method": "exact",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-3, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-3, 1.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

        if param["booster"] in ["gbtree", "dart"]:
            param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
            param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
            param["eta"] = trial.suggest_float("eta", 1e-4, 1.0, log=True)
            param["gamma"] = trial.suggest_float("gamma", 1e-4, 1.0, log=True)
            param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    
        if param["booster"] == "dart":
            param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
            param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
            param["rate_drop"] = trial.suggest_float("rate_drop", 1e-3, 1.0, log=True)
            param["skip_drop"] = trial.suggest_float("skip_drop", 1e-3, 1.0, log=True)

        boost = XGBRegressor(**param,  n_jobs=-1)

        score = np.mean((boost.fit(X_train.values, y_train).predict(X_test.values)-y_test)**2)
        return (score)

if __name__ == '__main__':
    sampler = optuna.samplers.TPESampler(
        #multivariate = True,
        #group = True
    )
    study = optuna.create_study(direction='minimize', 
                                storage=optuna_db_path, 
                                study_name='boost_complete_run_3', 
                                sampler = sampler,
                                load_if_exists=True)
    study.optimize(objective,
                   n_trials= 1000)
    

    print("Study statistics: ")

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

"learning_rate": trial.suggest_float('learning_rate', 0.01, 0.3, log=True),

"tree_method": "exact",

"booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),

"lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),

"alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),

"subsample": trial.suggest_float("subsample", 0.2, 1.0),

"colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),

In [ ]:
boost = XGBRegressor().fit(X_train.values, y_train.values)
test_mse = np.mean((boost.predict(X_test) - y_test)**2)
test_mse

In [ ]:
Boost_opt = XGBRegressor(**trial.params).fit(X_train.values, y_train.values)

In [ ]:
test_mse = np.mean((Boost_opt.predict(X_test) - y_test)**2)
test_mse

### with pruning

In [ ]:
train_X, valid_X, train_y, valid_y = train_test_split(X_train, y_train, test_size = 0.3)
valid_y

In [ ]:
optuna_db_path = "sqlite:////Users/mingyanggao/Desktop/SRF/my_optuna_study.db"

def objective(trial):
        #max_seconds = 180
        param = {
        "learning_rate": trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        "tree_method": "exact",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

        if param["booster"] in ["gbtree", "dart"]:
            param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
            param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
            param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
            param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    
        if param["booster"] == "dart":
            param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
            param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
            param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
            param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
        
        boost = boost = XGBRegressor(**param)

        #pruning
        #for step in range(50):
            #start_time = time.time()

        #    train_X, valid_X, train_y, valid_y = train_test_split(X_train, y_train, test_size = 0.3)
        #    boost.fit(train_X.values, train_y.values)

            #if time.time() - start_time > max_seconds:
            #    raise optuna.TrialPruned()

        #    intermediate_value = np.mean((boost.predict(valid_X.values) - valid_y.values)**2)
            
        #    trial.report(intermediate_value,step)
        #    if trial.should_prune():
        #        raise optuna.TrialPruned()
        
        #if not pruned
        scores = skm.cross_val_score(boost, X_train.values, y_train.values, n_jobs=-1, cv=3, scoring='neg_mean_squared_error')
        actual_mse_scores = [-s for s in scores]
        mean_mse = np.mean(actual_mse_scores)
        report_cross_validation_scores(trial, scores.tolist())
        return (mean_mse)
        

if __name__ == '__main__':
    study = optuna.create_study(direction='minimize', 
                                storage=optuna_db_path, 
                                study_name='boost_official_5%', 
                                load_if_exists=True)
    study.optimize(objective,
                   n_trials= 200)
    
    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    

In [ ]:
params_best = {
    'learning_rate': 0.1453396227151677,
    'booster': 'dart',
    'lambda': 3.8703489993224236e-08,
    'alpha': 0.003032430517597912,
    'subsample': 0.790961129756034,
    'colsample_bytree': 0.7273439201999695,
    'max_depth': 7,
    'min_child_weight': 8,
    'eta': 1.7488976584656106e-07,
    'gamma': 0.031707710814832854,
    'grow_policy': 'lossguide',
    'sample_type': 'uniform',
    'normalize_type': 'forest',
    'rate_drop': 6.726506333721989e-08,
    'skip_drop': 0.001512580988907405

}

In [ ]:
best = XGBRegressor(**params_best)
best.fit(X_train.values, y_train.values)
x = np.mean((best.predict(X_test.values)-y_test.values)**2)
x

evaluation

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300, step=50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 50, step=1)

    rf = RandomForestRegressor(
        n_estimators = n_estimators,
        min_samples_split = min_samples_split,
        n_jobs = -1
    )

    #for step in range(50):
    #    train_X, valid_X, train_y, valid_y = train_test_split(X_train, y_train, test_size = 0.3)
    #    rf.fit(train_X.values, train_y.values)
    #    intermediate_value = np.mean((rf.predict(valid_X.values) - valid_y.values)**2)
    #    trial.report(intermediate_value,step)
    #    if trial.should_prune():
    #        raise optuna.TrialPruned()

    scores = skm.cross_val_score(rf, X_train, y_train, n_jobs=-1, cv=3, scoring='neg_mean_squared_error')
    actual_mse_scores = [-s for s in scores]
    mean_mse = np.mean(actual_mse_scores)
    report_cross_validation_scores(trial, scores.tolist())
    return (mean_mse)


if __name__ == '__main__':
    study = optuna.create_study(direction='minimize', 
                                storage=optuna_db_path, 
                                study_name='rf_complete',  # Changed study name to avoid distribution conflict
                                load_if_exists=True)
    study.optimize(objective,
                   n_trials= 20)
    
    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
     



## FC NN

### Torch with no Optuna

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler

In [ ]:
# DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
DEVICE = torch.device('mps')
DEVICE

In [ ]:
a = X_train.shape[1]
X_train_numeric = X_train.astype(np.float32)
X_test_numeric = X_test.astype(np.float32)

X_train_tensor = torch.tensor(X_train_numeric.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test_numeric.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [ ]:
X_train_tensor = X_train_tensor.to(DEVICE)
y_train_tensor = y_train_tensor.to(DEVICE)
X_test_tensor = X_test_tensor.to(DEVICE)
y_test_tensor = y_test_tensor.to(DEVICE)

In [ ]:
criterion = nn.MSELoss()

In [ ]:
class FC(nn.Module):
    def __init__(self):
        super(FC, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(a, 64),   
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)    # Output: single value (price)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
NN = FC()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(NN.parameters(), lr = 0.01)

for epoch in range(100):
    NN.train()
    optimizer.zero_grad()
    output = NN(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

    print(f'Epoch: {epoch}; Loss: {loss.item()}')

In [ ]:
NN.eval()
with torch.no_grad():
    predictions = NN(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)
    print(f"Test MSE: {test_loss.item():.4f}")

### Torch with Optuna

In [ ]:
DEVICE = torch.device('mps')

In [ ]:
DEVICE

In [ ]:
#DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def define_model(trial):
    n_layers = trial.suggest_int("n_layers", 1, 10, step = 1)
    layers = []

    in_features = a

    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features

    layers.append(nn.Linear(in_features, 1))
    return nn.Sequential(*layers)

def train_NN(model, optimizer, X_train_NN, y_train_NN):
    criterion = nn.MSELoss()
    model.train()
    optimizer.zero_grad()
    output = model(X_train_NN)
    loss = criterion(output, y_train_NN)
    loss.backward()
    optimizer.step()

def eval_NN(model, X_test_NN, y_test_NN):
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_NN)
        test_loss = criterion(predictions, y_test_NN)
        return test_loss

def objective(trial):
    X_train_NN = X_train_tensor
    y_train_NN = y_train_tensor
    X_test_NN = X_test_tensor
    y_test_NN = y_test_tensor

    model = define_model(trial).to(DEVICE)
    optimizer = torch.optim.Adam(
        model.parameters(), trial.suggest_float("lr", 1e-5, 1e-1, log=True))

    for epoch in range(50):
        train_NN(model, optimizer, X_train_NN, y_train_NN)
    MSE = eval_NN(model, X_test_NN, y_test_NN)
    return MSE

In [ ]:
DEVICE

In [ ]:
PYTORCH_MPS_HIGH_WATERMARK_RATIO=1

In [ ]:
study = optuna.create_study(directions= ["minimize"],
                            storage=optuna_db_path, 
                            study_name='NN_complete_run', 
                            load_if_exists=True)
                            
study.optimize(objective,
               n_trials=20) 

In [ ]:
study.best_params

In [ ]:
class best_NN(nn.Module):
    def __init__(self):
        super(best_NN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(a, 72),
            nn.ReLU(),
            nn.Dropout(0.41393624755510494),
            nn.Linear(72, 62),
            nn.ReLU(),
            nn.Dropout(0.30910461566591313),
            nn.Linear(62, 36),
            nn.ReLU(),
            nn.Dropout(0.326896643945087),
            nn.Linear(36, 1)                      
        )
    def forward(self, x):
        return self.net(x)

NN = best_NN().to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(NN.parameters(), lr=0.05046040510249565)

for epoch in range(1000):
    NN.train()
    optimizer.zero_grad()
    output = NN(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

NN.eval()
with torch.no_grad():
    predictions = NN(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)
    print(f"Test MSE: {test_loss.item():.4f}")

## Best Torch updated by Kaggle

In [ ]:
class best_NN(nn.Module):
    def __init__(self):
        super(best_NN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(a, 73),
            nn.ReLU(),
            nn.Dropout(0.3120482244670788),
            nn.Linear(73, 99),
            nn.ReLU(),
            nn.Dropout(0.3480728906394902),
            nn.Linear(99, 85),
            nn.ReLU(),
            nn.Dropout(0.249992788940722),
            nn.Linear(85, 14),
            nn.ReLU(),
            nn.Dropout(0.22961914614037873),
            nn.Linear(14,1)
        )
    def forward(self, x):
        return self.net(x)

NN = best_NN().to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(NN.parameters(), lr=0.01129180147251508)

for epoch in range(200):
    NN.train()
    optimizer.zero_grad()
    output = NN(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

NN.eval()
with torch.no_grad():
    predictions = NN(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)
    print(f"Test MSE: {test_loss.item():.4f}")

# Ensemble

## Bagging

bagging with XGBoost, the actual run will use 100 estimator while in the test only 10 will be used

In [ ]:
opt_params =     {'learning_rate': 0.11904245065065563,
                    'booster': 'gbtree',
                    'lambda': 4.572570120030754e-07,
                    'alpha': 0.4604968728612804,
                    'subsample': 0.9216847013051581,
                    'colsample_bytree': 0.7745972825238412,
                    'max_depth': 9,
                    'min_child_weight': 5,
                    'eta': 4.1884110093375496e-07,
                    'gamma': 1.3519060667913771e-08,
                    'grow_policy': 'lossguide'}
Boost_opt = XGBRegressor(**opt_params)

In [ ]:
bagging_XGBoost = BaggingRegressor(
    Boost_opt,
    n_estimators = 100,
    max_samples  = 0.8,
    bootstrap = True,
    random_state = 1,
    oob_score = True,
    verbose = 2
)

bagging_XGBoost.fit(X_train.values, y_train.values)

In [ ]:
MSE_predict_bagging_boosting  = np.mean((bagging_XGBoost.predict(X_test.values) - y_test)**2)
MSE_predict_bagging_boosting

## Voting/Averaging

In [ ]:
from skorch import NeuralNetRegressor
net = NeuralNetRegressor(
    module = NN,
    max_epochs=100,
    criterion = nn.MSELoss(),
    optimizer = torch.optim.Adam,
    lr=0.05046040510249565
)

In [ ]:
voting = VotingRegressor(
    estimators = [
        ('Boosting', Boost_opt),
        ('Net', net)],
    verbose = 2)
    
voting = voting.fit(X_train_tensor.cpu().numpy(), y_train_tensor.cpu().numpy())

evaluation

In [ ]:
_ = X_test_tensor.cpu().numpy().ravel()

In [ ]:
MSE_predict_voting = np.mean((voting.predict(X_test_tensor.cpu().numpy()) - y_test)**2)
MSE_predict_voting

# Undone

using the default single layer for final estimator

In [ ]:
stacked = [
    ('Boosting', XGBoost_opt),
    ('DTR', DTR_opt),
    ('NN',NN_opt)
]

stack = StackingRegressor(
    estimators = stacked,
    final_estimator = None,
    cv = 5,
    verbose = 2
)

stack.fit(X_train.values, y_train.values)

evaluation

In [ ]:
MSE_predict_stacking       = np.mean((stack.predict(X_test.values) - y_test)**2)
MSE_predict_stacking

using a voting regressor as a customized final layer

In [ ]:
alt_final = VotingRegressor(
    estimators = [
        ('Ridge',RidgeCV()),
        ('RF', RandomForestRegressor())],
    verbose = 2
        )

stack_vote = StackingRegressor(
    estimators = stacked,
    final_estimator = alt_final,
    cv = 5,
    verbose = 2
    )

stack_vote.fit(X_train.values, y_train.values)

evaluation

In [ ]:
MSE_predict_stack_vote       = np.mean((stack_vote.predict(X_test.values) - y_test)**2)
MSE_predict_stack_vote

## SHAP: variable importance

## Neural Network + Kenel SHAP

### Forceplot

since the train set is small enough, using only k = 2

in the actual run k = 50

In [ ]:
background = shap.kmeans(X_train, 50)

in the trail run the test set is small enough, so producing SHAP values for all

in the actual run, would only need a portion of test set's SHAP values

In [ ]:
# background = X_train

In [ ]:
NN_explainer = shap.KernelExplainer(NN_opt.predict, background)

In [ ]:
shap_values = NN_explainer.shap_values(X_test.iloc[0:1000], verbose =2)

In [ ]:
shap.force_plot(
  NN_explainer.expected_value, 
  shap_values[0:1000], 
  X_test.iloc[0:1000])

In [ ]:
shap.force_plot(
  NN_explainer.expected_value, 
  shap_values[0:10], 
  X_test.iloc[0:10]
  )

### Beehive Plot

In [ ]:
NN_SHAP_summary_plt = shap.summary_plot(shap_values[0:1000], X_test[0:1000])

### Dependence plot

Again in the trail run all data in X_test is used

In the actual run only a portion is needed

In [ ]:
for f in list(X_test.columns):
    shap.dependence_plot(f , shap_values[0:1000], X_test[0:1000])

## XGBoost + Tree SHAP

### Forceplot

XGBoost does not support K-means background

selecting a portion to predict as calibration

using all of X_train in trial run

using 50 is enough in the actual run

In [ ]:
background_boosting = X_train.iloc[0:500].astype(float)

In [ ]:
Boosting_explainer = shap.TreeExplainer(XGBoost_opt, background_boosting)

trial: actual

actual: all

In [ ]:
shap_values_boosting = Boosting_explainer.shap_values(X_test.iloc[0:1000])

In [ ]:
shap_values_boosting = Boosting_explainer.shap_values(X_test.iloc[0:1000])
shap.force_plot(Boosting_explainer.expected_value, shap_values_boosting[0:1000], X_test.iloc[0:1000])

### Beehive plot

In [ ]:
NN_SHAP_summary_plt = shap.summary_plot(shap_values_boosting[0:1000], X_test[0:1000])

### Dependence plot

In [ ]:
for f in list(X_test.columns):
    shap.dependence_plot(f, shap_values_boosting[0:1000], X_test[0:1000])

## DTR + Tree SHAP

### Forceplot

In [ ]:
background_DTR = X_train.iloc[0:50].astype(float)

In [ ]:
DTR_explainer = shap.TreeExplainer(DTR_opt,background_DTR)

In [ ]:
shap_values_DTR = DTR_explainer.shap_values(X_test.iloc[0:1000])
shap.force_plot(DTR_explainer.expected_value, shap_values_DTR[0:1000], X_test.iloc[0:1000])

### Beehive plot

In [ ]:
shap.summary_plot(shap_values_DTR[0:1000], X_test.iloc[0:1000])

### Dependence plot

In [ ]:
for f in list(X_test.columns):
    shap.dependence_plot(f, shap_values_DTR[0:1000], X_test.iloc[0:1000])

## NN Bagging + Kernel SHAP

### Forceplot

In [ ]:
bagging_NN_explainer   = shap.KernelExplainer(bagging_NN.predict, background)
shap_values_bagging_NN = bagging_NN_explainer.shap_values(X_test.iloc[0:1000])

In [ ]:
shap.force_plot(bagging_NN_explainer.expected_value, shap_values_bagging_NN[0:1000], X_test.iloc[0:1000])

### Beehive plot

In [ ]:
shap.summary_plot(shap_values_bagging_NN[0:1000], X_test[0:1000])

### Dependence plot

In [ ]:
for f in list(X_test.columns):
    shap.dependence_plot(f, shap_values_bagging_NN[0:1000], X_test.iloc[0:1000])